In [69]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from sklearn.utils import shuffle


In [70]:
train = pd.read_csv('/home/az/dev/atvisor/nlp/classifier/input/main_res.csv',encoding="ISO-8859-1")
test = pd.read_csv('/home/az/dev/atvisor/nlp/classifier/data/test.csv', encoding = "utf-8")

In [71]:
print('Total rows in test is {}'.format(len(test)))  
print('Total rows in train is {}'.format(len(train)))
cols_target = list(train)[2:]
print(train[cols_target].sum())

Total rows in test is 106
Total rows in train is 1026
category_access stairs                               3
category_alarms                                      5
category_ankle and wrist weights                     1
category_anti slip mats                              6
category_aprons and bibs                             5
category_arm slings                                  4
category_arm supports                                1
category_back support for bath                       1
category_bands and elastics                          1
category_bath lift pommel                            1
category_bath seats                                  5
category_bathing sponges                             1
category_bathing supports and stabilisers            1
category_bathrobes and towels                        3
category_bathroom caddy                              1
category_bathtub shelves                             1
category_bathtub slings                              1
category_ba

In [72]:
data = train[cols_target]   

In [73]:
def clean_text(text):
    text = text.lower()
    text = text.strip(' ')
    return text


In [74]:
test['title'] = test['title'].map(lambda com : clean_text(com))
train['title'] = train['title'].map(lambda com : clean_text(com))


In [75]:
X = train.title

In [76]:
test_X = test.title

In [77]:
print(X.shape, test_X.shape)


(1026,) (106,)


In [78]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=5000,stop_words='english')
vect

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [79]:
X_dtm = vect.fit_transform(X)

In [80]:
X_dtm

<1026x1007 sparse matrix of type '<class 'numpy.float64'>'
	with 2530 stored elements in Compressed Sparse Row format>

In [81]:
test_X_dtm = vect.transform(test_X)


In [85]:
test_X_dtm



<106x1007 sparse matrix of type '<class 'numpy.float64'>'
	with 210 stored elements in Compressed Sparse Row format>

In [88]:
# import and instantiate the Logistic Regression model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(C=12.0)

# create submission file
sample_submission = pd.read_csv('/home/az/dev/atvisor/nlp/classifier/data/sample_submission.csv', encoding = "utf-8")
for label in cols_target:
    #print('... Processing {}'.format(label))
    y = train[label]
    # train the model using X_dtm & y
    logreg.fit(X_dtm, y)
    # compute the training accuracy
    y_pred_X = logreg.predict(X_dtm)
    #print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
    # compute the predicted probabilities for X_test_dtm
    test_y_prob = logreg.predict_proba(test_X_dtm)[:,1]
    submission_binary[label] = test_y_prob
submission_binary.to_csv('submission_binary.csv',index=False)

In [89]:
submission_binary.describe()

,id,category_access stairs,category_alarms,category_ankle and wrist weights,category_anti slip mats,category_aprons and bibs,category_arm slings,category_arm supports,category_back support for bath,category_bands and elastics,...,category_weighted blankets,category_wet wipes,category_whisks and brushes,category_white boards,category_whiteboard reminders,category_wipers,category_wrist hold-down and supports,category_wrist orthoses,category_zesters,category_zipper pullers
count,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,...,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000
mean,52.500000,0.002029,0.004245,0.001399,0.001852,0.004214,0.003398,0.003103,0.001844,0.001111,...,0.001189,0.001272,0.001632,0.002493,0.001034,0.001111,0.002275,0.002077,0.001111,0.001798
std,30.743563,0.001317,0.021531,0.002346,0.001939,0.015232,0.015606,0.016369,0.004007,0.000036,...,0.000046,0.001664,0.000083,0.000279,0.000031,0.000036,0.008728,0.007543,0.000036,0.000096
min,0.000000,0.001452,0.001013,0.001002,0.001142,0.002230,0.001531,0.000818,0.000886,0.001016,...,0.000993,0.001016,0.001365,0.001358,0.000952,0.001016,0.001178,0.001180,0.001016,0.001504
25%,26.250000,0.001616,0.002066,0.001067,0.001418,0.002572,0.001680,0.001032,0.000972,0.001083,...,0.001159,0.001083,0.001576,0.002396,0.001011,0.001083,0.001264,0.001307,0.001083,0.001727
50%,52.500000,0.001702,0.002185,0.001101,0.001486,0.002755,0.001775,0.001064,0.001006,0.001115,...,0.001196,0.001115,0.001634,0.002542,0.001032,0.001115,0.001313,0.001355,0.001115,0.001810
75%,78.750000,0.001752,0.002297,0.001128,0.001524,0.002922,0.001849,0.001089,0.001028,0.001143,...,0.001228,0.001145,0.001708,0.002690,0.001061,0.001143,0.001355,0.001398,0.001143,0.001882
max,105.000000,0.010557,0.223825,0.022952,0.017167,0.159544,0.162379,0.164204,0.025061,0.001160,...,0.001247,0.018242,0.001745,0.002795,0.001078,0.001160,0.090736,0.079001,0.001160,0.001927
